## 1. Import Library

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
import torch
from torch.utils.data import TensorDataset, DataLoader

# Konfigurasi tampilan
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
sns.set_style("whitegrid")

print("Library berhasil diimpor.")

## 2. Load Dataset

In [ ]:
try:
    df = pd.read_csv('https://github.com/erlanggadewasakti/Prinsip-Sains-Data/releases/download/prod/sa-psd-dataset.csv')
    print(f"Dataset berhasil dimuat: {len(df)} baris, {len(df.columns)} kolom")
except Exception as e:
    print(f"Error: {e}")

## 3. Inspeksi Dataset

In [ ]:
print("Sample Data:")
display(df.head())

print("\nInfo Dataset:")
df.info()

print("\nStatistik Deskriptif:")
display(df.describe(include='all'))

## 4. Identifikasi Missing Values

In [ ]:
missing_values = df.isnull().sum()
missing_percentage = (df.isnull().sum() / len(df)) * 100

missing_df = pd.DataFrame({
    'Kolom': missing_values.index,
    'Jumlah Missing': missing_values.values,
    'Persentase (%)': missing_percentage.values
})

print("Analisis Missing Values:")
print(missing_df[missing_df['Jumlah Missing'] > 0])

if missing_df['Jumlah Missing'].sum() == 0:
    print("\nTidak ada missing values.")
else:
    print(f"\nTotal missing: {missing_df['Jumlah Missing'].sum()}")

    # Visualisasi jika ada missing values
    plt.figure(figsize=(10, 6))
    missing_cols = missing_df[missing_df['Jumlah Missing'] > 0]
    plt.barh(missing_cols['Kolom'], missing_cols['Persentase (%)'])
    plt.xlabel('Persentase Missing (%)')
    plt.title('Distribusi Missing Values')
    plt.tight_layout()
    plt.show()

In [ ]:
# Strategi: Drop baris dengan missing values pada kolom input/output
df_clean = df.copy()

if df_clean.isnull().sum().sum() > 0:
    initial_rows = len(df_clean)
    df_clean = df_clean.dropna(subset=['input', 'output'], how='any')
    dropped_rows = initial_rows - len(df_clean)
    print(f"Baris dihapus: {dropped_rows}")
    print(f"Sisa data: {len(df_clean)} baris")
else:
    print("Tidak ada missing values.")

print(f"\nVerifikasi - Missing values: {df_clean.isnull().sum().sum()}")

## 5. Identifikasi Data Duplikat

In [ ]:
duplicates_all = df_clean.duplicated().sum()
duplicates_input = df_clean.duplicated(subset=['input']).sum()

print(f"Duplikat (semua kolom): {duplicates_all}")
print(f"Duplikat (kolom input): {duplicates_input}")

if duplicates_input > 0:
    print("\nContoh data duplikat:")
    duplicate_samples = df_clean[df_clean.duplicated(subset=['input'], keep=False)].sort_values('input').head(10)
    display(duplicate_samples[['input', 'output']])

In [ ]:
initial_rows = len(df_clean)
df_clean = df_clean.drop_duplicates(subset=['input'], keep='first')
removed_duplicates = initial_rows - len(df_clean)

print(f"Duplikat dihapus: {removed_duplicates}")
print(f"Total data: {len(df_clean)} baris")

df_clean = df_clean.reset_index(drop=True)
print("Index direset.")

## 6. Text Cleaning

In [ ]:
def advanced_text_cleaning(text):
    """Membersihkan teks dari URL, HTML tags, karakter khusus, dan whitespace berlebihan"""
    if not isinstance(text, str):
        return str(text) if text is not None else ""

    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # Hapus URL
    text = re.sub(r'<.*?>', '', text)  # Hapus HTML tags
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Hapus karakter khusus
    text = re.sub(r'\s+', ' ', text).strip()  # Normalisasi whitespace

    return text

df_clean['cleaned_input'] = df_clean['input'].apply(advanced_text_cleaning)

print("Perbandingan Teks:")
for idx, row in df_clean[['input', 'cleaned_input']].head(3).iterrows():
    print(f"\n[{idx + 1}] Original : {row['input'][:100]}...")
    print(f"    Cleaned  : {row['cleaned_input'][:100]}...")

print(f"\nText cleaning selesai untuk {len(df_clean)} baris.")

## 7. Ekstraksi dan Normalisasi Label Sentimen

In [ ]:
# Ekstraksi label dari kolom output
df_clean['sentiment'] = df_clean['output'].str.replace(r'^[A-E]:\s*', '', regex=True)

print("Distribusi Sebelum Normalisasi:")
print(df_clean['sentiment'].value_counts())

# Normalisasi: gabungkan very positive/negative dengan positive/negative
sentiment_normalization = {
    'very positive': 'positive',
    'very negative': 'negative',
    'positive': 'positive',
    'negative': 'negative',
    'neutral': 'neutral'
}

df_clean['sentiment'] = df_clean['sentiment'].map(sentiment_normalization)

print("\nDistribusi Setelah Normalisasi:")
print(df_clean['sentiment'].value_counts())
print("\nProporsi:")
print(df_clean['sentiment'].value_counts(normalize=True))

print("\nSample Hasil:")
display(df_clean[['output', 'sentiment', 'cleaned_input']].head(5))

## 8. Visualisasi Distribusi Sentimen

In [ ]:
sentiment_colors = {'positive': '#2ecc71', 'negative': '#e74c3c', 'neutral': '#3498db'}

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Count plot
sns.countplot(x='sentiment', data=df_clean, hue='sentiment', palette=sentiment_colors, legend=False, ax=axes[0])
axes[0].set_title('Distribusi Sentimen', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Kategori')
axes[0].set_ylabel('Jumlah')
for container in axes[0].containers:
    axes[0].bar_label(container, fmt='%d')

# Pie chart
sentiment_counts = df_clean['sentiment'].value_counts()
colors = [sentiment_colors[label] for label in sentiment_counts.index]
axes[1].pie(sentiment_counts.values, labels=sentiment_counts.index, autopct='%1.1f%%',
            colors=colors, startangle=90)
axes[1].set_title('Proporsi Sentimen', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

# Analisis imbalance
majority_class = sentiment_counts.idxmax()
minority_class = sentiment_counts.idxmin()
imbalance_ratio = sentiment_counts.max() / sentiment_counts.min()

print(f"Mayoritas: {majority_class} ({sentiment_counts.max()} data)")
print(f"Minoritas: {minority_class} ({sentiment_counts.min()} data)")
print(f"Rasio imbalance: {imbalance_ratio:.2f}:1")

if imbalance_ratio > 3:
    print("⚠️  Dataset imbalanced - perlu oversampling!")
else:
    print("✓ Dataset relatif seimbang.")

## 9. Encoding Label Sentimen

In [ ]:
sentiment_mapping = {'positive': 0, 'neutral': 1, 'negative': 2}
df_clean['sentiment_encoded'] = df_clean['sentiment'].map(sentiment_mapping)

print("Mapping Sentimen:")
for sentiment, code in sentiment_mapping.items():
    count = (df_clean['sentiment_encoded'] == code).sum()
    print(f"{sentiment:10s} -> {code} ({count:6d} data)")

print("\nSample Data:")
display(df_clean[['cleaned_input', 'sentiment', 'sentiment_encoded']].head(5))

assert df_clean['sentiment_encoded'].isnull().sum() == 0, "Ada nilai yang tidak termap!"
print("\n✓ Encoding berhasil.")

## 10. Analisis Panjang Teks

In [ ]:
df_clean['text_length_chars'] = df_clean['cleaned_input'].str.len()
df_clean['text_length_words'] = df_clean['cleaned_input'].str.split().str.len()

print("Statistik Panjang (Karakter):")
print(df_clean['text_length_chars'].describe())

print("\nStatistik Panjang (Kata):")
print(df_clean['text_length_words'].describe())

# Visualisasi
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].hist(df_clean['text_length_chars'], bins=50, color='skyblue', edgecolor='black', alpha=0.7)
axes[0].axvline(df_clean['text_length_chars'].median(), color='red', linestyle='--',
                label=f"Median: {df_clean['text_length_chars'].median():.0f}")
axes[0].set_xlabel('Jumlah Karakter')
axes[0].set_ylabel('Frekuensi')
axes[0].set_title('Distribusi Panjang Teks (Karakter)', fontweight='bold')
axes[0].legend()

axes[1].hist(df_clean['text_length_words'], bins=50, color='lightcoral', edgecolor='black', alpha=0.7)
axes[1].axvline(df_clean['text_length_words'].median(), color='blue', linestyle='--',
                label=f"Median: {df_clean['text_length_words'].median():.0f}")
axes[1].set_xlabel('Jumlah Kata')
axes[1].set_ylabel('Frekuensi')
axes[1].set_title('Distribusi Panjang Teks (Kata)', fontweight='bold')
axes[1].legend()

plt.tight_layout()
plt.show()

## 11. Split Data (Train/Val/Test)

In [ ]:
RANDOM_STATE = 42

# Split training (70%) dan temporary (30%)
df_train, df_temp, _, _ = train_test_split(
    df_clean, df_clean['sentiment_encoded'],
    test_size=0.3, random_state=RANDOM_STATE, stratify=df_clean['sentiment_encoded']
)

# Split temporary menjadi validation (15%) dan testing (15%)
df_val, df_test, _, _ = train_test_split(
    df_temp, df_temp['sentiment_encoded'],
    test_size=0.5, random_state=RANDOM_STATE, stratify=df_temp['sentiment_encoded']
)

print(f"Training   : {len(df_train)} baris ({len(df_train)/len(df_clean)*100:.1f}%)")
print(f"Validation : {len(df_val)} baris ({len(df_val)/len(df_clean)*100:.1f}%)")
print(f"Testing    : {len(df_test)} baris ({len(df_test)/len(df_clean)*100:.1f}%)")

# Verifikasi stratifikasi
print("\nDistribusi Sentimen:")
print("Train     :", df_train['sentiment'].value_counts(normalize=True).sort_index().to_dict())
print("Validation:", df_val['sentiment'].value_counts(normalize=True).sort_index().to_dict())
print("Test      :", df_test['sentiment'].value_counts(normalize=True).sort_index().to_dict())
print("\n✓ Stratifikasi berhasil.")

## 12. Oversampling Training Set

**Catatan**: Oversampling hanya pada training set untuk mencegah data leakage.

In [ ]:
majority_count = df_train['sentiment'].value_counts().max()

print(f"Target per kelas: {majority_count} samples")
print("\nSebelum oversampling:")
print(df_train['sentiment'].value_counts().sort_index())

df_train_oversampled = pd.DataFrame()

for sentiment_label in df_train['sentiment'].unique():
    sentiment_df = df_train[df_train['sentiment'] == sentiment_label]
    current_count = len(sentiment_df)

    if current_count < majority_count:
        oversampled_df = sentiment_df.sample(n=majority_count, replace=True, random_state=RANDOM_STATE)
        print(f"{sentiment_label:10s}: {current_count:5d} -> {len(oversampled_df):5d} (oversampled)")
    else:
        oversampled_df = sentiment_df
        print(f"{sentiment_label:10s}: {current_count:5d} (unchanged)")

    df_train_oversampled = pd.concat([df_train_oversampled, oversampled_df], ignore_index=True)

# Shuffle data
df_train_oversampled = df_train_oversampled.sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)

print("\nSetelah oversampling:")
print(df_train_oversampled['sentiment'].value_counts().sort_index())
print(f"\nTotal: {len(df_train)} -> {len(df_train_oversampled)} baris (+{len(df_train_oversampled)-len(df_train)})")
print("✓ Oversampling selesai.")

## 13. Visualisasi Perbandingan Oversampling

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Sebelum oversampling
df_train['sentiment'].value_counts().sort_index().plot(
    kind='bar', ax=axes[0], color=['#e74c3c', '#3498db', '#2ecc71'],
    edgecolor='black', alpha=0.7
)
axes[0].set_title('SEBELUM Oversampling', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Sentimen')
axes[0].set_ylabel('Jumlah')
axes[0].set_xticklabels(['Negative', 'Neutral', 'Positive'], rotation=0)
for container in axes[0].containers:
    axes[0].bar_label(container, fmt='%d')

# Setelah oversampling
df_train_oversampled['sentiment'].value_counts().sort_index().plot(
    kind='bar', ax=axes[1], color=['#e74c3c', '#3498db', '#2ecc71'],
    edgecolor='black', alpha=0.7
)
axes[1].set_title('SETELAH Oversampling', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Sentimen')
axes[1].set_ylabel('Jumlah')
axes[1].set_xticklabels(['Negative', 'Neutral', 'Positive'], rotation=0)
for container in axes[1].containers:
    axes[1].bar_label(container, fmt='%d')

plt.tight_layout()
plt.show()

## 14. Load BERT Tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

print(f"Tokenizer: bert-base-uncased")
print(f"Vocab size: {tokenizer.vocab_size:,}")
print(f"Max length: {tokenizer.model_max_length}")
print("✓ Tokenizer loaded.")

## 15. Analisis Panjang Token

In [ ]:
token_lengths = [
    len(tokenizer.encode(str(text), add_special_tokens=True, truncation=True, max_length=512))
    for text in df_train_oversampled['cleaned_input']
]

print("Statistik Panjang Token:")
print(f"Min     : {np.min(token_lengths)}")
print(f"Max     : {np.max(token_lengths)}")
print(f"Mean    : {np.mean(token_lengths):.2f}")
print(f"Median  : {np.median(token_lengths):.0f}")
print(f"P95     : {np.percentile(token_lengths, 95):.0f}")
print(f"P99     : {np.percentile(token_lengths, 99):.0f}")

# Visualisasi
plt.figure(figsize=(12, 6))
sns.histplot(token_lengths, bins=50, kde=True, color='steelblue', edgecolor='black', alpha=0.6)
plt.axvline(np.mean(token_lengths), color='red', linestyle='--',
            label=f'Mean: {np.mean(token_lengths):.0f}', linewidth=2)
plt.axvline(np.percentile(token_lengths, 95), color='orange', linestyle='--',
            label=f'P95: {np.percentile(token_lengths, 95):.0f}', linewidth=2)
plt.xlabel('Jumlah Token')
plt.ylabel('Frekuensi')
plt.title('Distribusi Panjang Token', fontweight='bold')
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

# Tentukan MAX_LENGTH
suggested = int(np.percentile(token_lengths, 95))
MAX_LENGTH = min(suggested, 512)
if MAX_LENGTH > 400:
    MAX_LENGTH = 330  # Nilai praktis

print(f"\nSuggested MAX_LENGTH: {suggested}")
print(f"Chosen MAX_LENGTH: {MAX_LENGTH}")
print(f"Coverage: {(np.array(token_lengths) <= MAX_LENGTH).sum() / len(token_lengths) * 100:.1f}%")

## 16. Tokenisasi Final

In [ ]:
def tokenize_data(texts, tokenizer, max_length):
    """Tokenisasi teks dengan padding dan truncation"""
    return tokenizer(
        list(texts),
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors='pt'
    )

# Tokenisasi semua dataset
X_train_tokenized = tokenize_data(df_train_oversampled['cleaned_input'], tokenizer, MAX_LENGTH)
X_val_tokenized = tokenize_data(df_val['cleaned_input'], tokenizer, MAX_LENGTH)
X_test_tokenized = tokenize_data(df_test['cleaned_input'], tokenizer, MAX_LENGTH)

print(f"MAX_LENGTH: {MAX_LENGTH} tokens\n")
print(f"Training   : {X_train_tokenized['input_ids'].shape}")
print(f"Validation : {X_val_tokenized['input_ids'].shape}")
print(f"Testing    : {X_test_tokenized['input_ids'].shape}")
print("\n✓ Tokenisasi selesai.")

## 17. Data Dictionary

In [ ]:
data_dict = pd.DataFrame({
    'Kolom': ['input', 'output', 'cleaned_input', 'sentiment', 'sentiment_encoded', 'text_length_chars', 'text_length_words'],
    'Tipe': ['string', 'string', 'string', 'string', 'int64', 'int64', 'int64'],
    'Deskripsi': [
        'Teks original dari dataset',
        'Label output dengan format prefix (A:, B:, dst)',
        'Teks yang sudah dibersihkan dari noise',
        'Label sentimen kategorikal (positive/negative/neutral)',
        'Label sentimen numerik (0: positive, 1: neutral, 2: negative)',
        'Jumlah karakter teks bersih',
        'Jumlah kata teks bersih'
    ],
    'Contoh': ['Amazing product!', 'A: very positive', 'Amazing product', 'positive', '0', '16', '2']
})

print("Data Dictionary:")
display(data_dict)

## 18. Summary Preprocessing

In [ ]:
print("LAPORAN PREPROCESSING")
print("=" * 60)

print("\n📊 Statistik Dataset:")
print(f"  Original         : {len(df):,} baris")
print(f"  Duplikat dihapus : {removed_duplicates:,} baris")
print(f"  Data bersih      : {len(df_clean):,} baris")

print("\n📂 Data Split:")
print(f"  Training (ori)   : {len(df_train):,} baris")
print(f"  Training (over)  : {len(df_train_oversampled):,} baris")
print(f"  Validation       : {len(df_val):,} baris")
print(f"  Testing          : {len(df_test):,} baris")

print("\n🔄 Transformasi:")
print(f"  Text cleaning    : ✓")
print(f"  Label encoding   : ✓ (3 kelas)")
print(f"  Oversampling     : ✓ (ratio 1:1:1)")
print(f"  Tokenization     : ✓ (BERT, max_length={MAX_LENGTH})")

print("\n📈 Distribusi Sentimen (Training Oversampled):")
for sent in ['negative', 'neutral', 'positive']:
    count = (df_train_oversampled['sentiment'] == sent).sum()
    pct = count / len(df_train_oversampled) * 100
    print(f"  {sent.capitalize():10s}: {count:6,} ({pct:5.1f}%)")

print("\n" + "=" * 60)
print("✓ PREPROCESSING SELESAI - DATASET SIAP")
print("=" * 60)

## 19. Ekspor Dataset (Opsional)

In [ ]:
output_file = 'sentiment_dataset_cleaned.csv'

columns_to_save = ['input', 'output', 'cleaned_input', 'sentiment', 'sentiment_encoded', 'text_length_chars', 'text_length_words']

df_clean[columns_to_save].to_csv(output_file, index=False)

print(f"✓ Dataset saved: {output_file}")
print(f"  Rows: {len(df_clean):,}")
print(f"  Cols: {len(columns_to_save)}")

In [ ]:
# Konversi tokenized data ke PyTorch tensors
input_ids_train = X_train_tokenized['input_ids']
attention_mask_train = X_train_tokenized['attention_mask']
token_type_ids_train = X_train_tokenized['token_type_ids']
labels_train = torch.tensor(df_train_oversampled['sentiment_encoded'].values.astype(int))

input_ids_val = X_val_tokenized['input_ids']
attention_mask_val = X_val_tokenized['attention_mask']
token_type_ids_val = X_val_tokenized['token_type_ids']
labels_val = torch.tensor(df_val['sentiment_encoded'].values.astype(int))

input_ids_test = X_test_tokenized['input_ids']
attention_mask_test = X_test_tokenized['attention_mask']
token_type_ids_test = X_test_tokenized['token_type_ids']
labels_test = torch.tensor(df_test['sentiment_encoded'].values.astype(int))

# Buat TensorDataset
train_dataset = TensorDataset(input_ids_train, attention_mask_train, token_type_ids_train, labels_train)
val_dataset = TensorDataset(input_ids_val, attention_mask_val, token_type_ids_val, labels_val)
test_dataset = TensorDataset(input_ids_test, attention_mask_test, token_type_ids_test, labels_test)

# Buat DataLoader
BATCH_SIZE = 32
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Batch size: {BATCH_SIZE}\n")
print(f"Train batches : {len(train_dataloader)}")
print(f"Val batches   : {len(val_dataloader)}")
print(f"Test batches  : {len(test_dataloader)}")
print("\n✓ DataLoader siap untuk training.")